# Import and data pre-processing

In [ ]:
import os
os.chdir("C:\\Users\\")

In [ ]:
pwd

In [ ]:
import pandas as pd
import datetime as dt

In [ ]:
df = pd.read_excel("xxx.xlsx")
df['Dato'] = df['Dato'].map(lambda x: x.rstrip(' UTC'))
df['Dato'].head()

In [ ]:
df['Dato'] = pd.to_datetime(df['Dato'], format="%d.%m.%Y %H:%M:%S").dt.date

In [ ]:
df['Dato'] = pd.to_datetime(df['Dato'])

In [ ]:
# df.dtypes
# df.head()
df["Dato"].head()

# Cohort Analysis

In [ ]:
# First:
# Define a function that will parse the date
def get_day(x): return dt.datetime(x.year, x.month, x.day) 

# Create InvoiceDay column
df['InvoiceDay'] = df['Dato'].apply(get_day)

# Group by CustomerID and select the InvoiceDay value
grouping = df.groupby('Gjennomføre')['InvoiceDay'] 

# Assign a minimum InvoiceDay value to the dataset
df['CohortDay'] = grouping.transform('min')

# View the top 5 rows
print(df.head())


In [ ]:
def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

# Get the integers for date parts from the `InvoiceDay` column
invoice_year, invoice_month, invoice_day = get_date_int(df, "InvoiceDay")

# Get the integers for date parts from the `CohortDay` column
cohort_year, cohort_month, cohort_day = get_date_int(df, "CohortDay")


In [ ]:
invoice_month.head()

In [ ]:
# Calculate difference in years
years_diff = invoice_year - cohort_year

# Calculate difference in months
months_diff = invoice_month - cohort_month

# Calculate difference in days
days_diff = invoice_day - cohort_day

# Extract the difference in days from all previous values
df['CohortIndex'] = years_diff * 365 + months_diff * 30 + days_diff + 1
print(df.head())



In [ ]:
#Create a groupby object as grouping DataFrame with this command: 
grouping = df.groupby(['CohortDay', 'CohortIndex'])

# Count the number of unique values per customer ID
cohort_data = grouping['Gjennomføre'].apply(pd.Series.nunique).reset_index()

# Create a pivot 
cohort_counts = cohort_data.pivot(index='CohortDay', columns='CohortIndex', values='Gjennomføre')
cohort_counts

In [ ]:
# Select the first column and store it to cohort_sizes
cohort_sizes = cohort_counts.iloc[:,0]

# Divide the cohort count by cohort sizes along the rows
retention = cohort_counts.divide(cohort_sizes, axis=0)
retention = retention *100
retention.round(0)

In [ ]:
# Import seaborn package as sns
from matplotlib import pyplot as plt
import seaborn as sns

# Initialize an 8 by 6 inches plot figure
plt.figure(figsize=(15, 15))

# Add a title
plt.title('Average Return Rate by Monthly Cohorts')

# Create the heatmap
sns.heatmap(data=retention, annot=True, cmap='Blues')
plt.show()


# Recency, Frequency analysis

In [ ]:
#Let's create a hypothetical snapshot_day data so we can do the exercise as if we're doing analysis recently.
snapshot_date = max(df.Dato) + dt.timedelta(days=1)

# Aggregate data on a customer level
datamart = df.groupby(['Gjennomføre']).agg({
    'Dato': lambda x: (snapshot_date - x.max()).days,
    'Gjennomføre': 'count'})

# Rename columns for easier interpretation
datamart.rename(columns = {'Dato': 'Recency',
                           'Gjennomføre': 'Frequency'}, inplace=True)
# Check the first rows
datamart.head()


In [ ]:
# Recency quartile
# Create labels
r_labels = range(4, 0, -1)
# Assign these labels to four equal percentile groups
r_quartiles = pd.qcut(datamart['Recency'], 4, labels = r_labels)
# Create new columns R
datamart = datamart.assign(R = r_quartiles.values)
datamart.head()

In [ ]:
# Frequency and Monetary quartiles
# Create labels
f_labels = range(1,5)

# Assign these labels to four equal percentile groups
f_quartiles = pd.qcut(datamart['Frequency'], 4, labels = f_labels)

# Create new columns F and M
datamart = datamart.assign(F = f_quartiles.values)
datamart.head()

In [ ]:
def join_rfm(x): return str(x['R']) + str(x['F']) 
datamart['RFM_Segment'] = datamart.apply(join_rfm, axis=1)
datamart['RFM_Score'] = datamart[['R','F']].sum(axis=1)

datamart.head()

In [ ]:
#Filtering on RFM segments
# Largest RFM segments
datamart.groupby('RFM_Segment').size().sort_values(ascending=False)[:10]

In [ ]:
# Select and view the 15 best customers by RFM Score
datamart.sort_values('RFM_Score', ascending=False).head(15)

In [ ]:
# Select and view the 15 worst customers by RFM Score
datamart.sort_values('RFM_Score').head(15)

In [ ]:
datamart.groupby('RFM_Score').agg({
    'Recency': 'mean',
    'Frequency': ['mean', 'count']}).round(1)


In [ ]:
# Grouping into named segments
def segment_me(df):
    if df['RFM_Score'] >= 8:
        return '1. Gold'
    elif (df['RFM_Score'] >= 5) and (df['RFM_Score'] < 8):
        return '2. Silver'
    else:
        return '3. Bronze'
# Create a new variable RFM_Level
datamart['RFM_Level'] = datamart.apply(segment_me, axis=1)
# Calculate average values for each RFM_Level, and return a size of each segment 
rfm_level_agg = datamart.groupby('RFM_Level').agg({
    'Recency': 'mean',
    'Frequency': ['mean', 'count']
}).round(1)

# Print the aggregated dataset
print(rfm_level_agg)


# Data pre-processing for K-means clustering
Key k-means assumptions

•	Symmetric distribution of variables (not skewed)

    o	Skew removed with logarithmic transformation (Log = only on non-negative values)
    
•	Variables with same average values

•	Variables with same variance


In [ ]:
datamart_rfm = datamart[['Recency', 'Frequency']]


In [ ]:
datamart_rfm.head()

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
sns.distplot(datamart['Frequency'])
plt.show()


In [ ]:
import numpy as np
frequency_log = np.log(datamart['Frequency'])
sns.distplot(frequency_log)
plt.show()

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
sns.distplot(datamart['Recency'])
plt.show()

In [ ]:
frequency_log= np.log(datamart['Recency'])
sns.distplot(frequency_log)
plt.show()

In [ ]:
# Coding the sequence
#Unskew the data with log transformation
import numpy as np
datamart_log = np.log(datamart_rfm)

# Normalize the variables with StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(datamart_log)

# Scale and center the data
#Store it separately for clustering
datamart_normalized = scaler.transform(datamart_log)

# Create a pandas DataFrame
datamart_normalized = pd.DataFrame(datamart_normalized, index= datamart_log.index, columns= datamart_log.columns)


In [ ]:
datamart_normalized.describe()

# 4.	Practical implementation of k-means clustering
Key steps

•	Data pre-processing

•	Choosing a number of clusters

•	Running k-means clustering on pre-processed data

•	Analyzing average RFM values of each cluster

Methods to define the number of clusters

•	Visual methods - elbow criterion

•	Mathematical methods - silhouette coefficient

•	Experimentation and interpretation


In [ ]:
# Elbow criterion method
# Plot the number of clusters against within-cluster 
#       sum-of-squared-errors (SSE) - sum of squared distances from every data point to their cluster center
# Identify an "elbow" in the plot
# Elbow - a point representing an "optimal" number of clusters
# Import key libraries
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt

 # Fit KMeans and calculate SSE for each *k*
sse = {}
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=1)
    kmeans.fit(datamart_normalized)
    sse[k] = kmeans.inertia_ # sum of squared distances to closest cluster center



 # Plot SSE for each *k*
plt.title('The Elbow Method')
plt.xlabel('k'); plt.ylabel('SSE')
sns.pointplot(x=list(sse.keys()), y=list(sse.values()))
plt.show()
# Best to choose the point on elbow, or the next point

In [ ]:
# Import KMeans from sklearn library and initialize it as kmeans
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=1)

# Compute k-means clustering on pre-processed data
kmeans.fit(datamart_normalized)

# Extract cluster labels from labels_ attribute
cluster_labels = kmeans.labels_
#4.2. Analyzing average RFM values of each cluster
# Create a cluster label column in the original DataFrame:
datamart_rfm_k3 = datamart_rfm.assign(Cluster = cluster_labels)

# Calculate average RFM values and segment size for each cluster:
datamart_rfm_k3.groupby(['Cluster']).agg({
    'Recency': 'mean',
    'Frequency': ['mean', 'count']
}).round(0)


### 4.4.	Profile and interpret segments

Approaches to build customer personas

•	Summary statistics for each cluster e.g. average RFM values

•	Snake plots (from market research

•	Relative importance of cluster attributes compared to population


In [ ]:
# Snake plots to understand and compare segments
# Market research technique to compare different segments
# Visual representation of each segment's attributes

# Need to first normalize data (center & scale)
# Plot each cluster's average normalized values of each attribute
# Transform datamart_normalized as DataFrame and add a Cluster column
datamart_normalized = pd.DataFrame(datamart_normalized, 
                                   index=datamart_rfm.index, 
                                   columns=datamart_rfm.columns)
datamart_normalized['Cluster'] = datamart_rfm_k3['Cluster']

# Melt the data into a long format so RFM values and metric names are stored in 1 column each
datamart_melt = pd.melt(datamart_normalized.reset_index(), 
                    id_vars=['CustomerID', 'Cluster'],
                    value_vars=['Recency', 'Frequency', 'MonetaryValue'], 
                    var_name='Attribute', 
                    value_name='Value')

# Visualize the snake plot
plt.title('Snake plot of standardized variables')
sns.lineplot(x="Attribute", y="Value", hue='Cluster', data=datamart_melt)


In [ ]:
# Relative importance of segment attributes
# Useful technique to identify relative importance of each segment's attribute
# Calculate average values of each cluster
# Calculate average values of population
# Calculate importance score by dividing them and subtracting 1
cluster_avg = datamart_rfm_k3.groupby(['Cluster']).mean()
population_avg = datamart_rfm.mean()
relative_imp = cluster_avg / population_avg - 1

# Analyze and plot relative importance
# The further a ratio is from 0, the more important that attribute is for a segment relative to the total population.
relative_imp.round(2)

# Plot a heatmap for easier interpretation: 
plt.figure(figsize=(12, 4))
plt.title('Relative importance of attributes')
sns.heatmap(data=relative_imp, annot=True, fmt='.2f', cmap='RdYlGn')
plt.show()


# Regression for explanatory purpose

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import statsmodels.formula.api as smf
import statsmodels.api as sm # import statsmodels 
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [ ]:
df['weekday'] = df['Dato'].dt.dayofweek

In [ ]:
cat_vars=["weekday","Klassefisering"]
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(df[var], prefix=var)
    data1=df.join(cat_list)
    df=data1
cat_vars=["weekday","Klassefisering"]
data_vars=df.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]


In [ ]:
lst = ['xxx','xxx','xxx'....]

lst2 = ['xxx','xxx','xxx'....]

df = pd.merge(datamart,df[lst],left_index=True, right_on = "Gjennomføre",  how="left")


In [ ]:
X = df[lst2]

y = df["Recency"]

model = sm.OLS(y, X).fit()
predictions = model.predict(X)

model.summary()

In [ ]:
# Saturday and Klassefisering_VIEW_RAW_DATA are standard variables

reg = smf.ols(formula = "Recency ~ 'xxx','xxx','xxx'...", data = df).fit()
reg.summary()

In [ ]:
# Saturday and Klassefisering_VIEW_RAW_DATA are standard variables

reg = smf.ols(formula = "Frequency ~ 'xxx','xxx','xxx'...", data = df).fit()
reg.summary()